### Анализ игр NBA

Подготовил Григорий Крюков

Источник данных: https://www.kaggle.com/datasets/nathanlauga/nba-games

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
df = pd.read_csv("games.csv")

In [3]:
df["date"] = [datetime.strptime(df.GAME_DATE_EST[i], "%Y-%m-%d") for i in range(len(df))]

In [4]:
df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,date
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0,2022-03-12
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1,2022-03-12
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0,2022-03-12
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1,2022-03-12
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0,2022-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25791,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1,2014-10-06
25792,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0,2014-10-06
25793,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1,2014-10-06
25794,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1,2014-10-05


In [5]:
df = df.sort_values('date')
df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,date
18433,2003-10-05,10300001,Final,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,...,41.0,1610612742,85.0,0.447,0.500,0.250,20.0,38.0,1,2003-10-05
18432,2003-10-06,10300002,Final,1610612763,1610612749,2003,1610612763,105.0,0.494,0.618,...,48.0,1610612749,94.0,0.427,0.700,0.154,20.0,43.0,1,2003-10-06
18425,2003-10-07,10300010,Final,1610612764,1610612752,2003,1610612764,104.0,0.506,0.677,...,45.0,1610612752,86.0,0.380,0.852,0.188,19.0,37.0,1,2003-10-07
18424,2003-10-07,10300006,Final,1610612747,1610612744,2003,1610612747,NaN,NaN,NaN,...,NaN,1610612744,NaN,NaN,NaN,NaN,NaN,NaN,0,2003-10-07
18427,2003-10-07,10300004,Final,1610612742,1610612753,2003,1610612742,99.0,0.488,0.667,...,52.0,1610612753,89.0,0.408,0.727,0.350,15.0,34.0,1,2003-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2022-03-12,22101010,Final,1610612762,1610612758,2021,1610612762,134.0,0.558,0.710,...,44.0,1610612758,125.0,0.500,0.857,0.394,27.0,33.0,1,2022-03-12
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0,2022-03-12
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1,2022-03-12
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1,2022-03-12


In [6]:
df.HOME_TEAM_ID.iloc[1]

1610612763

In [7]:
# ID команд в таблицах дома и на выезде совпадают
sorted(df.HOME_TEAM_ID.unique()) == sorted(df.VISITOR_TEAM_ID.unique())

True

In [8]:
team_id = sorted(df.HOME_TEAM_ID.unique())
len(team_id)

30

In [9]:
N = 30
prev_win = [1 for i in range(N)] # Выиграна ли предыдущая игра
TM = [np.zeros((2, 2)) for i in range(N)] # Матрицы переходов
k = 1610612737 # Параметр, вычитание которого из id команды приводит к порядковому номеру
# Проходимся циклом с конца таблицы (в хронологическом порядке), заполняем соответствующие значения матриц переходов
for i in range(len(df)):
    home = df.HOME_TEAM_ID.iloc[i] - k
    visitor = df.VISITOR_TEAM_ID.iloc[i] - k
    if df.HOME_TEAM_WINS.iloc[i] == 1:
        TM[home][1 - prev_win[home]][0] += 1
        TM[visitor][1 - prev_win[visitor]][1] += 1
        prev_win[home] = 1
        prev_win[visitor] = 0
    else:
        TM[home][1 - prev_win[home]][1] += 1
        TM[visitor][1 - prev_win[visitor]][0] += 1
        prev_win[home] = 0
        prev_win[visitor] = 1

In [10]:
# Промежуточная проверка на правильность: сумма всех значений в матрицах переходов в два раза больше числа игр
S = 0
for i in range(N):
    S += TM[i].sum()
assert S == 2 * len(df) 
# Для каждой из команд модуль разницы между числом побед после поражения и числом поражений после побед не может превосходить 1
for i in range(N):
    assert np.abs(TM[i][0][1] - TM[i][1][0]) <= 1

In [11]:
# Теперь нам осталось получить "настоящие" оценки матриц переходов
# Для этого все строки матриц TM нормируем так, чтобы сумма по строкам была равна 1
for i in range(N):
    TM[i][0] /= TM[i][0].sum()
    TM[i][1] /= TM[i][1].sum()

In [12]:
# Полученные матрицы переходов
TM

[array([[0.488228  , 0.511772  ],
        [0.44745395, 0.55254605]]),
 array([[0.59545005, 0.40454995],
        [0.50933998, 0.49066002]]),
 array([[0.58555556, 0.41444444],
        [0.43155452, 0.56844548]]),
 array([[0.48425197, 0.51574803],
        [0.43219405, 0.56780595]]),
 array([[0.51963048, 0.48036952],
        [0.48998822, 0.51001178]]),
 array([[0.5758483 , 0.4241517 ],
        [0.56515957, 0.43484043]]),
 array([[0.57525773, 0.42474227],
        [0.53446034, 0.46553966]]),
 array([[0.59816887, 0.40183113],
        [0.5156658 , 0.4843342 ]]),
 array([[0.59044715, 0.40955285],
        [0.52755906, 0.47244094]]),
 array([[0.55095185, 0.44904815],
        [0.47789725, 0.52210275]]),
 array([[0.58557589, 0.41442411],
        [0.44976636, 0.55023364]]),
 array([[0.57899807, 0.42100193],
        [0.55050505, 0.44949495]]),
 array([[0.4981685 , 0.5018315 ],
        [0.45758929, 0.54241071]]),
 array([[0.42530488, 0.57469512],
        [0.38391039, 0.61608961]]),
 array([[0.48404255,

In [13]:
is_win_lover = 0
for i in range(N):
    if TM[i][0][0] > TM[i][1][0]:
        is_win_lover += 1
is_win_lover

29

In [14]:
win_lose_diff = 0
for i in range(N):
    win_lose_diff += TM[i][0][0] - TM[i][1][0]
win_lose_diff / N * 100

6.378878377185229

Для 29 команд из 30 вероятность выиграть после победы больше, чем вероятность выиграть после поражения.

В среднем вероятность победы в текущей игре при победе в прошлой игре на 6.38% выше, чем вероятность победы в текущей игре при поражении в прошлой игре.

Это подтверждает состоятельность нашего предположения о психологических эффектах.

#### Силы команд

Базовый подход: доля выигранных матчей.

In [15]:
w = np.zeros(N)
prev_win = [1 for i in range(N)] # Выиграна ли предыдущая игра
A = [np.zeros((2, 2)) for i in range(N)] # Матрицы переходов
k = 1610612737 # Параметр, вычитание которого из id команды приводит к порядковому номеру
# Проходимся циклом с конца таблицы (в хронологическом порядке), заполняем соответствующие значения матриц переходов
for i in range(len(df)):
    home = df.HOME_TEAM_ID.iloc[i] - k
    visitor = df.VISITOR_TEAM_ID.iloc[i] - k
    if df.HOME_TEAM_WINS.iloc[i] == 1:
        A[home][1 - prev_win[home]][0] += 1
        A[visitor][1 - prev_win[visitor]][1] += 1
        prev_win[home] = 1
        prev_win[visitor] = 0
    else:
        A[home][1 - prev_win[home]][1] += 1
        A[visitor][1 - prev_win[visitor]][0] += 1
        prev_win[home] = 0
        prev_win[visitor] = 1
        
# Теперь поделим число выигранных матчей на число всех матчей
for i in range(N):
    w[i] = A[i].sum(axis=1)[0] / A[i].sum()

In [16]:
w

array([0.46647399, 0.55733186, 0.5107832 , 0.45656081, 0.50495627,
       0.57126568, 0.55779183, 0.56203545, 0.56357388, 0.51618497,
       0.52044818, 0.56721311, 0.47755102, 0.4004884 , 0.44183314,
       0.40256567, 0.44791054, 0.52036718, 0.45147929, 0.51665693,
       0.49853372, 0.41005451, 0.64105379, 0.52561888, 0.51515152,
       0.53962704, 0.49853544, 0.43506494, 0.48242075, 0.40103159])

In [17]:
# Сортируем команды от слабейшей к сильнейшей
np.argsort(w)

array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
        9, 19, 17, 10, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64)

Стационарное распределение матрицы.

In [18]:
def get_stat(p, q):
    return q/(1-p+q), (1-p)/(1-p+q)

In [19]:
get_stat(0.5, 0.5)

(0.5, 0.5)

In [20]:
get_stat(0.7, 0.5)

(0.625, 0.37500000000000006)

In [21]:
get_stat(0.5, 0.3)

(0.37499999999999994, 0.625)

In [22]:
# Проверка на стабильность результатов в зависимости от нумерации команд:
for i in range(1000):
    p = np.random.random()
    q = np.random.random()
    assert np.round(get_stat(p, q)[0] - get_stat(1-q, 1-p)[1], 4) == 0

Теперь посчитаем полученную метрику для модели с психологическим эффектом.

$$u(i) = \frac{p_i + q_i - 1}{p_i q_i}$$

In [23]:
def func_u(p, q):
    return (p + q - 1) / (p * q)

u = np.zeros(N)
for i in range(N):
    p = TM[i][0][0]
    q = TM[i][1][0]
    u[i] = func_u(p, q)

In [24]:
u

array([-0.29441624,  0.34551495,  0.06770929, -0.39922432,  0.03777778,
        0.43327556,  0.35686192,  0.36904762,  0.37883731,  0.10956809,
        0.13419118,  0.40629532, -0.19408178, -1.16845878, -0.54395604,
       -1.09048886, -0.45384615,  0.15106383, -0.44489084,  0.11014447,
       -0.01138952, -1.05452357,  0.66452462,  0.16645054,  0.10677618,
        0.26614481, -0.01091703, -0.67179143, -0.14072539, -1.09642857])

In [25]:
# Сортируем команды от слабейшей к сильнейшей
np.argsort(u)

array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
        9, 19, 10, 17, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64)

Сравним сортировки w_i, u_i.

In [26]:
np.argsort(w), np.argsort(u)

(array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
         9, 19, 17, 10, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64),
 array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
         9, 19, 10, 17, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64))

In [27]:
np.corrcoef(np.argsort(w), np.argsort(u))

array([[1.      , 0.978198],
       [0.978198, 1.      ]])

Добавим информацию о командах (название).

In [28]:
dt = pd.read_csv("teams.csv")

In [29]:
dt_sorted = dt.sort_values(by='TEAM_ID')
dt_sorted

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
28,0,1610612739,1970,2019,CLE,Cavaliers,1970,Cleveland,Quicken Loans Arena,20562.0,Dan Gilbert,Koby Altman,John Beilein,Canton Charge
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
5,0,1610612743,1976,2019,DEN,Nuggets,1976,Denver,Pepsi Center,19099.0,Stan Kroenke,Tim Connelly,Michael Malone,No Affiliate
29,0,1610612744,1946,2019,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors
6,0,1610612745,1967,2019,HOU,Rockets,1967,Houston,Toyota Center,18104.0,Tilman Fertitta,Daryl Morey,Mike D'Antoni,Rio Grande Valley Vipers
7,0,1610612746,1970,2019,LAC,Clippers,1970,Los Angeles,Staples Center,19060.0,Steve Ballmer,Michael Winger,Doc Rivers,Agua Caliente Clippers of Ontario


In [30]:
teams_name = list(dt_sorted["ABBREVIATION"])

In [31]:
is_eastern_conf = np.array([1,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,0,0,1,1,1])
len(is_eastern_conf), is_eastern_conf.sum()

(30, 15)

In [32]:
# Команды по силам
for i in np.argsort(u):
    print(teams_name[i])

MIN
CHA
NYK
SAC
WAS
BKN
ORL
PHI
NOP
ATL
MIL
DET
POR
MEM
CHI
CLE
TOR
LAC
PHX
LAL
IND
OKC
UTA
BOS
DEN
GSW
HOU
MIA
DAL
SAS


Попробуем просимулировать турнир

In [33]:
from scipy.stats import bernoulli

In [34]:
# Пока упрощенная схема турниров
def f(w1, w2):
    return 0

In [35]:
# Генерируем матч
def is_first_win(w1, w2, TM1, TM2, mood1, mood2, f):
    val = f(w1, w2) 
    p = val + (1 - val) * TM1[mood1][0]*TM2[mood2][1] / (TM1[mood1][0]*TM2[mood2][1] + TM1[mood1][1]*TM2[mood2][0])
    return bernoulli.rvs(p)

In [36]:
# Первый матч сезона 2018/19 записан в строке 20918
# Первый матч сезона 2018/19 записан в строке 22147
# Все сыграли по 82 игры, красота, всё верно

games = np.zeros(N)
k = 1610612737 # Параметр, вычитание которого из id команды приводит к порядковому номеру
for i in range(20918, 22148):
    home = df.HOME_TEAM_ID.iloc[i] - k
    visitor = df.VISITOR_TEAM_ID.iloc[i] - k
    games[home] += 1
    games[visitor] += 1
games

array([82., 82., 82., 82., 82., 82., 82., 82., 82., 82., 82., 82., 82.,
       82., 82., 82., 82., 82., 82., 82., 82., 82., 82., 82., 82., 82.,
       82., 82., 82., 82.])

In [54]:
def generate_tourn(w, TM, df, f):
    N = 30
    k = 1610612737 # Параметр, вычитание которого из id команды приводит к порядковому номеру
    mood = np.array([0 for i in range(N)])
    wins = np.array([0 for i in range(N)])
    for i in range(20918, 22148):
        h = df.HOME_TEAM_ID.iloc[i] - k
        v = df.VISITOR_TEAM_ID.iloc[i] - k
        res = is_first_win(w[h], w[v], TM[h], TM[v], mood[h], mood[v], f)
        if res == 1:
            mood[h] = 0
            mood[v] = 1
            wins[h] += 1
        else:
            mood[h] = 1
            mood[v] = 0
            wins[v] += 1
    return wins, mood

In [46]:
test_gen = generate_tourn(w, TM, df, f)[0]
test_gen.max(), test_gen.sum()
# Суммарно должно быть 82*30/2=1230 побед

(52, 1230)

In [47]:
test_gen.argsort()

array([21,  3, 18, 15,  9, 27,  8,  0, 14, 29, 26, 11, 24, 20, 12, 23,  4,
       25,  2, 19, 28, 10, 16, 13,  5,  7,  1, 17, 22,  6], dtype=int64)

In [114]:
def gen_tour(w1, w2, TM1, TM2, mood1, mood2, f):
    win1 = 0
    win2 = 0
    m1 = mood1
    m2 = mood2
    while win1 < 4 and win2 < 4:
        res = is_first_win(w1, w2, TM1, TM2, mood1, mood2, f)
        if res == 1:
            m1 = 0
            m2 = 1
            win1 += 1
        else:
            m1 = 1
            m2 = 0
            win2 += 1
    if win1 == 4:
        return 1
    return 0

def gen_play_off(w, TM, is_eastern_conf, df, f, wins, mood):
    N = 30
    ranking = wins.argsort()
    conf = [np.zeros((2, 15)), np.zeros((2, 15))]
    nums = [0, 0]
    for i in range(N-1, -1, -1):
        t = ranking[i]
        ind = is_eastern_conf[t]
        conf[ind][0][nums[ind]] = t
        conf[ind][1][nums[ind]] = mood[t]
        nums[ind] += 1
    conf = [c[:,:8] for c in conf] # Кто прошел в плей-офф
    finals = []
    for c in conf:
        q_final = np.zeros(4)
        best_pos, worse_pos = 0, 3
        for i in range(4):
            h = int(c[0][i])
            v = int(c[0][7-i])
            res = gen_tour(w[h], w[v], TM[h], TM[v], int(c[1][i]), int(c[1][7-i]), f)
            if res == 1:
                q_final[best_pos] = h
                best_pos += 1
            else:
                q_final[worse_pos] = h
                worse_pos -= 1
        sem_final = []
        for i in range(2):
            h = int(c[0][i])
            v = int(c[0][3-i])
            res = gen_tour(w[h], w[v], TM[h], TM[v], 0, 0, f)
            if res == 1:
                sem_final.append(h)
            else:
                sem_final.append(v)
        h = int(sem_final[0])
        v = int(sem_final[1])
        res = gen_tour(w[h], w[v], TM[h], TM[v], 0, 0, f)
        if res == 1:
            finals.append(h)
        else:
            finals.append(v)
    h = int(finals[0])
    v = int(finals[1])
    res = gen_tour(w[h], w[v], TM[h], TM[v], 0, 0, f)
    if res == 1:
        return h
    return v

In [115]:
wins, mood = generate_tourn(w, TM, df, f)
wins.argsort(), gen_play_off(w, TM, is_eastern_conf, df, f, wins, mood)

(array([29, 14, 27, 21, 18, 15, 16, 12,  3, 24,  0,  5, 20, 13, 28,  8, 17,
         6, 23,  7, 10, 25,  1, 22,  2,  9, 26,  4, 19, 11], dtype=int64),
 22)

In [123]:
np.random.seed(42)
champ = np.zeros(N)
for _ in tqdm(range(10000)):
    wins, mood = generate_tourn(w, TM, df, f)
    champ[gen_play_off(w, TM, is_eastern_conf, df, f, wins, mood)] += 1

100%|██████████| 10000/10000 [19:31<00:00,  8.54it/s]


In [124]:
champ

array([4.400e+01, 1.234e+03, 5.770e+02, 5.000e+00, 1.880e+02, 4.840e+02,
       3.710e+02, 6.310e+02, 5.200e+02, 1.210e+02, 2.460e+02, 9.880e+02,
       6.200e+01, 0.000e+00, 1.400e+01, 1.000e+00, 5.100e+01, 2.360e+02,
       3.500e+01, 1.350e+02, 2.700e+01, 0.000e+00, 2.978e+03, 2.360e+02,
       3.480e+02, 2.090e+02, 7.700e+01, 8.000e+00, 1.710e+02, 3.000e+00])

In [125]:
champ.argsort()

array([21, 13, 15, 29,  3, 27, 14, 20, 18,  0, 16, 12, 26,  9, 19, 28,  4,
       25, 17, 23, 10, 24,  6,  5,  8,  2,  7, 11,  1, 22], dtype=int64)

In [126]:
w.argsort()

array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
        9, 19, 17, 10, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64)